# 使用 Gurobi 進行放射治療優化

在本範例中，我們將使用 Gurobi 在 Python 中建模並解決放射治療優化問題。目標是最小化對健康組織的輻射暴露，同時確保腫瘤區域獲得足夠的輻射劑量以進行癌症治療。

**問題描述：**

- **決策變數：**
  - $x_1$, $x_2$: 連續變數，表示治療強度或幾何形狀。
  - **Binary variables (0/1):** 代表是否在特定位置（例如，近距離治療中的放射性種子植入）放置的決策。
- **目標函數：**
  $$
  \text{Minimize } Z = 0.4x_1 + 0.5x_2
  $$
  其中 $Z$ 代表治療影響（健康組織的損傷）的加權總和。

- **約束條件：**
  $$
  0.3x_1 + 0.1x_2 \leq 2.7
  $$
  $$
  0.5x_1 + 0.5x_2 = 6
  $$
  $$
  0.6x_1 + 0.4x_2 \geq 6
  $$
  - 非負性約束：$x_1 \geq 0$, $x_2 \geq 0$。

此模型為一個 **混合整數規劃**（MIP）問題，其中連續變數和二元變數結合使用來優化醫療治療計畫。這類模型特別適用於自動化系統，需進行離散決策，例如在近距離治療中植入種子的問題。

## 第一步：導入 Gurobi 並設置環境

Gurobi 是一個強大的優化求解器，能夠處理線性規劃、混合整數規劃等數學優化問題。我們導入 `gurobipy` 模組以訪問其功能和類別。


In [1]:
import gurobipy as gp
from gurobipy import GRB

## 第二步：創建新模型

我們創建一個新的模型物件，並將決策變數、目標函數和約束條件添加到其中。命名模型有助於識別它，特別是在處理多個模型時。


In [2]:
# 創建新模型
model = gp.Model('Radiation_Therapy_Optimization')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-31


## 第三步：定義決策變數

我們定義了以下決策變數：

- 連續變數 \( x_1 \) 和 \( x_2 \)，表示治療強度。
- 二元變數 \( y_1 \) 和 \( y_2 \)，表示放置決策（例如，是否在特定位置植入放射性種子）。

連續變數有下限 0，二元變數可以取值 0 或 1。


In [3]:
# 添加連續決策變數
x1 = model.addVar(name='x1', vtype=GRB.CONTINUOUS, lb=0)
x2 = model.addVar(name='x2', vtype=GRB.CONTINUOUS, lb=0)

# 添加二元決策變數
y1 = model.addVar(name='y1', vtype=GRB.BINARY)
y2 = model.addVar(name='y2', vtype=GRB.BINARY)

## 第四步：設置目標函數

目標函數的目的是最小化對健康組織影響的加權總和：

\[
\text{Minimize } Z = 0.4x_1 + 0.5x_2
\]

我們使用 `setObjective` 方法將此目標設置到模型中，並指定 `GRB.MINIMIZE` 表示我們要最小化目標函數。


In [4]:
# 設置目標函數
model.setObjective(0.4 * x1 + 0.5 * x2, GRB.MINIMIZE)

## 第五步：添加約束條件

我們使用 `addConstr` 方法將約束條件添加到模型中。

### 約束 1：

$$
0.3x_1 + 0.1x_2 \leq 2.7
$$

此約束限制了對某個敏感區域的輻射暴露。

### 約束 2：

$$
0.5x_1 + 0.5x_2 = 6
$$

此約束確保總治療強度達到所需的水平。

### 約束 3：

$$
0.6x_1 + 0.4x_2 \geq 6
$$

此約束確保腫瘤區域獲得足夠的輻射。

### 連結連續變數與二元變數：

如果我們希望建模為當二元變數 $y_i$ 為 1 時，連續變數 $x_i$ 才可以為正值（即進行治療），我們可以添加以下約束：

$$
x_i \leq M y_i \quad \text{for } i = 1, 2
$$

其中 $M$ 為一個足夠大的常數。

假設 $M = 10$，我們添加這些約束來連結 $x_i$ 和 $y_i$。


In [5]:
# 添加約束條件
model.addConstr(0.3 * x1 + 0.1 * x2 <= 2.7, name='Constraint1')
model.addConstr(0.5 * x1 + 0.5 * x2 == 6, name='Constraint2')
model.addConstr(0.6 * x1 + 0.4 * x2 >= 6, name='Constraint3')

# 連結約束條件 x_i 和 y_i
M = 10  # 大常數
model.addConstr(x1 <= M * y1, name='Linking_x1_y1')
model.addConstr(x2 <= M * y2, name='Linking_x2_y2')

<gurobi.Constr *Awaiting Model Update*>

## 第六步：優化模型

模型定義完成後，我們調用 `optimize` 方法來解決優化問題。Gurobi 使用先進的演算法來有效地找到最佳解。


In [6]:
# 優化模型
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B5046f)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 4 columns and 10 nonzeros
Model fingerprint: 0xe9022a0c
Variable types: 2 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [4e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+00]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 5.25 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.250000000000e+00, best bound 5.250000000000e+00, gap 0.0000%


## 第七步：顯示結果

在優化過程完成後，我們可以訪問決策變數的最佳值以及目標函數的值。我們先檢查模型的狀態是否為 `GRB.OPTIMAL`，以確保找到最佳解，然後再顯示結果。


In [7]:
# 檢查模型是否有最佳解
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f"  x1 (Treatment intensity 1) = {x1.x}")
    print(f"  x2 (Treatment intensity 2) = {x2.x}")
    print(f"  y1 (Placement decision 1) = {y1.x}")
    print(f"  y2 (Placement decision 2) = {y2.x}")
    print(f"  Minimum impact (Z) = {model.objVal}")
else:
    print("No optimal solution found.")

Optimal solution found:
  x1 (Treatment intensity 1) = 7.5
  x2 (Treatment intensity 2) = 4.5
  y1 (Placement decision 1) = 1.0
  y2 (Placement decision 2) = 1.0
  Minimum impact (Z) = 5.25


## 結果解讀

最佳解提供了治療強度和放置決策，這樣可以在最小化對健康組織的影響的同時，確保腫瘤區域獲得足夠的輻射劑量。二元變數 $y_1$ 和 $y_2$ 表示是否在特定位置進行治療。
